# Tests Nov 4

In [20]:
import pandas as pd
import numpy as np
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [21]:
from invesscience.joanna_merge import get_training_data 

In [18]:
companies = get_training_data(reference="a")

In [22]:
companies.head()

,id,normalized_name,category_code,founded_at,description,closed_at,exit,exit_date,country_code,state_code,...,degree_count,MBA_bool,cs_bool,phd_bool,founder_count,n_female_founders,female_ratio,mean_comp_founded_ever,mean_comp_founded_before,target
0,c:4304,texas instruments,semiconductor,1930-01-01,semiconductor Semiconducter and Device Manuf...,NaN,ipo,1978-01-13 00:00:00,USA,TX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,c:1349,move networks,software,NaT,software Move Networks is a streaming tech pla...,NaN,acquisition,1987-06-16 00:00:00,USA,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,c:4584,modern feed,games_video,2007-01-01,games_video [Modern Feed](http://www.moder...,NaN,acquisition,1987-06-16 00:00:00,USA,CA,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
3,c:16612,micron technology,semiconductor,1978-01-01,semiconductor Semiconductor Applications Mic...,NaN,ipo,1990-01-02 00:00:00,USA,ID,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,c:28,cisco,network_hosting,1984-01-01,network_hosting Cisco designs and sells ha...,NaN,ipo,1990-02-23 00:00:00,USA,CA,...,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1


In [13]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8448 entries, 0 to 8447
Data columns (total 29 columns):
id                           8448 non-null object
normalized_name              8448 non-null object
category_code                7952 non-null object
founded_at                   6651 non-null datetime64[ns]
description                  8431 non-null object
closed_at                    1004 non-null object
exit                         8448 non-null object
exit_date                    1815 non-null object
country_code                 7994 non-null object
state_code                   5238 non-null object
date_series_0                8448 non-null datetime64[ns]
participants_0               8448 non-null int64
raised_amount_usd_0          8448 non-null float64
timediff_founded_series_0    6651 non-null float64
phd                          3326 non-null float64
MBA                          3326 non-null float64
cs                           3326 non-null float64
graduate               

In [7]:
companies.mean_comp_founded_before.value_counts()

0.000000    3078
1.000000      79
0.500000      71
0.333333      44
0.250000      17
0.200000       9
2.000000       8
0.666667       5
1.500000       3
0.400000       2
0.800000       2
0.750000       1
4.000000       1
3.000000       1
0.600000       1
1.333333       1
0.571429       1
1.666667       1
0.375000       1
Name: mean_comp_founded_before, dtype: int64

In [10]:
exits ={"acquisition":1, "ipo":1, "no exit":0}
companies["target"] = companies.exit.map(exits)